In [3]:
#pip install torchtext==0.6.0

In [1]:
import numpy as np
import pandas as pa
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import time

#import torchtext

In [7]:
TEXT = torchtext.data.Field(lower=True, fix_length=200, batch_first=True)
LABEL = torchtext.data.Field(sequential=False)

In [8]:
#from torchtext.datasets import IMDB
train_data, test_data = IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


.data\imdb\aclImdb_v1.tar.gz: 100%|████████████████████████████████████████████████| 84.1M/84.1M [01:35<00:00, 883kB/s]


In [10]:
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

## 전처리

In [25]:
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)["text"]] #키값만 들고와서 그안에 애들 다 lower해
    text = [x.replace("<br", "") for x in text]
    text = ["".join(c for c in s if c not in string.punctuation) for s in text] #불용어 처리인데. 이렇게 안써요..
    text = [s for s in text if s]
    vars(example)["text"] = text

for example in test_data.examples:
    text = [x.lower() for x in vars(example)["text"]] 
    text = [x.replace("<br", "") for x in text]
    text = ["".join(c for c in s if c not in string.punctuation) for s in text]
    text = [s for s in text if s]
    vars(example)["text"] = text

In [27]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(42), split_ratio=0.8)

In [31]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None) #담아내는데 10000개, 최소10번은 나오는 애들로, vector매트리스로 표현하지말고.
#print(vars(TEXT.vocab))
LABEL.build_vocab(train_data)

In [32]:
BATCH_SIZE = 64
embeding_dim = 100
hidden_size = 300

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE
)

In [35]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz, hidden_size))

        for word in inputs: #input은 문장전체 에서 , word 각각의 단어마다 가중치값을 구해서
            ht = self.rnn(word, ht) #기억셀만 뽑아냄
        return ht

class LSTMCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(LSTMCell_Encoder, self).__init__()
        self.lstm = nn.LSTMCell(input_dim, hidden_size)

    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz, hidden_size))
        ct = torch.zeros((bz, hidden_size))
        for word in inputs: 
            ht = self.rnn(word, (ht, ct)) 
        return ht

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim) #len 즉 단어갯수 몇개가 오면 embeding_dim 100, 밖으로 나오는 벡터는 100개짜리임
        self.rnn = LSTMCell_Encoder(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x)
        return x

In [36]:
model = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

### 문제 변경 : 네이버 리뷰를 통한 감정분석 (0 or 1의 분류문제로 접근)

> condition: 1. ML 사용 안하고, 2. 한국어 처리하며, 3. LSTM을 colab으로.